In [1]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import pandas as pd

def load_from_tsfile(file_path, return_y=True):
    X, y = load_from_tsfile_to_dataframe(file_path)
    X = pd.DataFrame({i: pd.Series(x) for i, x in enumerate(X.iloc[:, 0])})
    if return_y:
        return X, y
    else:
        return X


# Test the function
dname= 'FaceDetection'
file_path = f'./datasets/{dname}/{dname}_TRAIN.ts'
X, y = load_from_tsfile(file_path)
print(X.shape, y.shape)



C:\Users\ui572274\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

(62, 5890) (5890,)


In [2]:
file_path = f'./datasets/{dname}/{dname}_TEST.ts'
X_test, y_test = load_from_tsfile(file_path)
print(X_test.shape, y_test.shape)

c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

(62, 3524) (3524,)


In [ ]:
X_train  = X.transpose()

In [ ]:
X_test = X_test.transpose()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
y = label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)


In [ ]:
num_classes = len(np.unique(y_test))

num_classes

2

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    param = {
        'objective': 'binary',  # or 'multiclass' for multi-class classification
        'metric': 'binary_logloss',  # or 'multi_logloss' for multi-class classification

        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    }

    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy



In [8]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



[I 2024-02-08 17:39:31,300] A new study created in memory with name: no-name-7244450d-26b8-43e9-bcaa-4381ce31ecb2
[I 2024-02-08 17:39:37,827] Trial 0 finished with value: 0.5059591373439274 and parameters: {'num_leaves': 227, 'learning_rate': 1.6991281748374848e-08, 'n_estimators': 875}. Best is trial 0 with value: 0.5059591373439274.
[I 2024-02-08 17:39:38,017] Trial 1 finished with value: 0.5204313280363224 and parameters: {'num_leaves': 20, 'learning_rate': 5.9378822195477025e-05, 'n_estimators': 131}. Best is trial 1 with value: 0.5204313280363224.
[I 2024-02-08 17:39:39,858] Trial 2 finished with value: 0.5 and parameters: {'num_leaves': 129, 'learning_rate': 1.8542119018625523e-08, 'n_estimators': 373}. Best is trial 1 with value: 0.5204313280363224.
[I 2024-02-08 17:39:40,861] Trial 3 finished with value: 0.5124858115777525 and parameters: {'num_leaves': 189, 'learning_rate': 0.3006211571460333, 'n_estimators': 227}. Best is trial 1 with value: 0.5204313280363224.
[I 2024-02-08 

KeyboardInterrupt: 